# Scraping & Databasing The New Yorker's 'Tables For Two'
Last Updated: 10/31/2016

### The code in this notebook does three things:
1. Scrape The New Yorker's Tables For Two restaurant reviews
2. Insert important information for each restaurant into a sqlite database
3. Generate a list of 'markers' so we can visualize these locations on the map


In [159]:
#Let's import some useful packages

from bs4 import BeautifulSoup
import requests
import urllib.request
import pandas
from geopy.geocoders import Nominatim
import sqlite3 as lite
geolocator = Nominatim()

## 1. Scraping The New Yorker (for the very first time)

Starting with the most recent restaurants, each 'page' contains 10 restaurants.
We grab the links, add them to our list, then move on to the next page.

In [ ]:
### iterate through all pages of The New Yorker tables-for-two history                                                                            
step = 1
RestLinks = []
while step < 80:
    page = 'http://www.newyorker.com/magazine/tables-for-two/page/' + str(step)

    r = urllib.request.urlopen(page).read()
    soup = BeautifulSoup(r, "lxml")
        
    #return a list of lists of restaurants from each page
    info = soup.findAll('a', {'itemprop':'name'})
    
    #extract only the link itself
    for link in info:
        # add links to RestLinks list:                                                                               
        RestLinks.append(link.get('href'))

    step +=1

### How many restaurants in total? What does each entry look like so far?

In [157]:
print('%d total restaurants' % len(RestLinks))
for i in range(0,6):
    print(RestLinks[i].strip())

805 total restaurants
http://www.newyorker.com/magazine/2016/11/07/dining-from-a-simpler-time
http://www.newyorker.com/magazine/2016/10/31/the-lucky-bees-unconventional-thai-street-food
http://www.newyorker.com/magazine/2016/10/24/pondicheris-nirvanic-dishes
http://www.newyorker.com/magazine/2016/10/17/1633-strange-and-wonderful-on-the-upper-east-side
http://www.newyorker.com/magazine/2016/10/10/a-disneyland-of-organic-delights-at-olmsted
http://www.newyorker.com/magazine/2016/10/03/dining-for-the-modern-herbivore


### Now, get information from each restaurant-specific page

For each restaurant in RestLinks, we need to beautiful soup and save:
1. Restaurant name
2. Street Address
3. Phone number
4. Text of the review
5. Date of the article 

Lastly, we use Google's geolocator API to extract the latitude and longitude

In [ ]:
# let's save our results to a .txt file, just in case
T4Trests = open('T4T_restaurant_db_102816.txt', 'w')

with open('key.txt') as f:                                    
    key = f.readline()

lead = 'https://maps.googleapis.com/maps/api/geocode/json?address='

info = []
count = 1
for article in RestLinks:                                                                                                     
    r = urllib.request.urlopen(article).read()
    soup = BeautifulSoup(r, "lxml")
    # we will also be adding our info here -- to put into a sqlite database
    rest_info = []

    # grab restaurant name(which may be different from name given in link)
    name = soup.findAll('h1', {'itemprop':'headline'})[0].string

    # grab street and telephone addresses
    street = "null"  
    tel = "null"
    lat = "null"
    lng = "null"

    # use google's geocode API to look up latitude and longitude
    try:
        addresses = soup.findAll('h2', {'itemprop':'alternativeHeadline'})[0].string
        street = addresses.split('(')[0]
        street_split = street.replace(" ", "+")
        place = street_split + ",New+York+City,+NY"
        query = (lead + place + key).strip()
        resp_json_payload = requests.get(query).json()
        
        lat = (str(resp_json_payload['results'][0]['geometry']['location']['lat']))
        lng = (str(resp_json_payload['results'][0]['geometry']['location']['lng']))
        tel = addresses.split('(')[1].strip(')')
    except:
        pass 


    # grab article date
    date = article.split('/')
    dt = date[4] + "/" + date[5] + "/" + date[6]

    #grab text for later meta-analysis
    div = soup.find_all('div',{'itemprop': 'articleBody'})
    for tag in div:
        text = tag.find_all('p')
    text_sum = ""
    for p in text:
        text_sum += str(p)

    rest_info = (count, name, article, street, str(lat), str(lng), tel, dt, text_sum)

    # save our info in memory for writing to database
    info.append(rest_info)
    # write results to a text file
    T4Trests.write(article + "\t" + name + "\t" + street + "\t" + str(lat) + "\t"+ str(lng) + "\t" + tel + "\t"+ dt + "\t"+ text_sum + "\n")
    count +=1
    
T4Trests.close()

## 2. Insert important information for each restaurant into a sqlite database

In [ ]:
# create a new database, T4T_070916.db that contains the table Rest_inf

con = lite.connect('T4T_070916.db')

with con:
    cur = con.cursor()
    cur.execute("CREATE TABLE Rest_inf(Id INTEGER PRIMARY KEY, Name TEXT, Article TEXT, Street TEXT, Latitude FLOAT, Longitude FLOAT, Telephone TEXT, Date TEXT, Text TEXT)");
    cur.executemany("INSERT INTO Rest_inf VALUES(?,?,?,?,?,?,?,?,?)", info)

## 3. Generate a list of 'markers' so we can visualize these locations on the map

Google's html (var locations) works best with a list of lists, where each list contains [name, lat, lng, index]
Currently, I am writing my list of lists to a .txt file, then copy and pasting into 'map.html' document

Would also like to update this to have:
1. Links to the review from the New Yorker (DONE)
2. Tags RE: food type

In [151]:
output = []
gmaps = []
with con:
    for row in cur.execute("SELECT Name, Latitude, Longitude, Id, Article FROM Rest_inf ORDER BY Date desc"):
        output.append(row)

for entry in output:
    try:
        lat = float(entry[1])
        lng = float(entry[2])
    except:
        lat = "null"       # a fair number of empty locations :(
        lng = "null"
    link = '<a href="' + entry[4] + '">' + entry[0] + '</a>'
    item = [link, lat, lng, int(entry[3])]
    gmaps.append(item)
    
gmaps_list = open('gmaps_list_102916.txt', 'w')
gmaps_list.write(str(gmaps))
gmaps_list.close()
